# 1. Imports

In [1]:
from impt_functions import *
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd 
import numpy as np
from fairlearn.reductions import ExponentiatedGradient, GridSearch, DemographicParity, EqualizedOdds, \
    TruePositiveRateParity, FalsePositiveRateParity, ErrorRateParity, BoundedGroupLoss
from fairlearn.metrics import *
from raiwidgets import FairnessDashboard

# 2. Prepare data

In [2]:
data = get_data('/home/mackenzie/git_repositories/delayedimpact/data/simData_oom10.csv')

      score  repay_probability  race  repay_indices
0       610              78.90     1              1
1       568              47.77     0              0
2       750              98.13     1              1
3       775              98.45     1              1
4       704              95.88     1              1
...     ...                ...   ...            ...
9995    832              98.99     1              1
9996    416              10.91     1              0
9997    444              14.63     1              0
9998    778              98.47     1              1
9999    738              97.68     1              1

[10000 rows x 4 columns]


In [3]:
X_train, X_test, y_train, y_test, race_train, race_test, sample_weight_train, sample_weight_test = prep_data(data=data, test_size=0.3, weight_index=1)

Sample weights are all equal.


# 3. Gaussian Naive Bayes classifier (Fairlearn used)

## Train GNB classifier + Collect Predictions
NOTE: atm sample_weight are all 1s

In [4]:
# Initialize classifier:
gnb = GaussianNB()

# Train the classifier:
model = gnb.fit(X_train, y_train, sample_weight_train)

# Make predictions with the classifier:
y_predict = model.predict(X_test)

### Evaluation of classifier overall

In [5]:
cm = confusion_matrix(y_test, y_predict)
print(cm)
print(classification_report(y_test, y_predict)) 
evaluation_outcome_rates(y_test, y_predict, sample_weight_test)

[[ 617  232]
 [ 188 1963]]
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       849
           1       0.89      0.91      0.90      2151

    accuracy                           0.86      3000
   macro avg       0.83      0.82      0.82      3000
weighted avg       0.86      0.86      0.86      3000

FNER=FN/(FN+TP)=  0.08740120874012088
FPER=FP/(FP+TN)=  0.27326266195524146
TNR=TN/(TN+FP)=  0.7267373380447585
TPR=TP/(FP+FN)=  0.9125987912598791


### Evaluation of classifier by race

In [6]:
evaluation_by_race(X_test, y_test, race_test, y_predict, sample_weight_test)

EVALUATION FOR BLACK GROUP
[[233   0]
 [122   4]]
              precision    recall  f1-score   support

           0       0.66      1.00      0.79       233
           1       1.00      0.03      0.06       126

    accuracy                           0.66       359
   macro avg       0.83      0.52      0.43       359
weighted avg       0.78      0.66      0.54       359

FNER=FN/(FN+TP)=  0.9682539682539683
FPER=FP/(FP+TN)=  0.0
TNR=TN/(TN+FP)=  1.0
TPR=TP/(FP+FN)=  0.031746031746031744

EVALUATION FOR WHITE GROUP
[[ 384  232]
 [  66 1959]]
              precision    recall  f1-score   support

           0       0.85      0.62      0.72       616
           1       0.89      0.97      0.93      2025

    accuracy                           0.89      2641
   macro avg       0.87      0.80      0.82      2641
weighted avg       0.88      0.89      0.88      2641

FNER=FN/(FN+TP)=  0.03259259259259259
FPER=FP/(FP+TN)=  0.37662337662337664
TNR=TN/(TN+FP)=  0.6233766233766234
TPR=TP/(FP+

### Delayed impact calculated by race

In [7]:
calculate_delayed_impact(X_test, y_test, y_predict, race_test)

The delayed impact of the black group is:  0.8356545961002786
The delayed impact of the white group is:  42.455509276789094


### Fairness Metric Evaluation of classifier

In [18]:
print_fairness_metrics(y_test, y_predict, race_test)

Selection Rate Overall:  0.7316666666666667
Selection Rate By Group:  sensitive_feature_0
0    0.0111421
1      0.82961
Name: selection_rate, dtype: object 

DP Difference:  0.8184679349322184
-->difference of 0 means that all groups have the same selection rate
DP Ratio: 0.013430480987681945
-->ratio of 1 means that all groups have the same selection rate 

EOD Difference:  0.9356613756613756
-->difference of 0 means that all groups have the same TN, TN, FP, and FN rates
EOD Ratio: 0.0
-->ratio of 1 means that all groups have the same TN, TN, FP, and FN rates rates


The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


## Exponentiated Gradient Reduction Alg for Adding Fairness Constraints

### Demographic Parity

In [8]:
add_contraint(model, 'DP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Exponentiated Gradient Reduction Alg is used here with  DP  as the fairness constraint.

Evaluation of  DP -constrained classifier overall:
[[ 615  234]
 [ 909 1242]]
              precision    recall  f1-score   support

           0       0.40      0.72      0.52       849
           1       0.84      0.58      0.68      2151

    accuracy                           0.62      3000
   macro avg       0.62      0.65      0.60      3000
weighted avg       0.72      0.62      0.64      3000

FNER=FN/(FN+TP)=  0.4225941422594142
FPER=FP/(FP+TN)=  0.2756183745583039
TNR=TN/(TN+FP)=  0.7243816254416962
TPR=TP/(FP+FN)=  0.5774058577405857


Evaluation of  DP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[114 119]
 [ 72  54]]
              precision    recall  f1-score   support

           0       0.61      0.49      0.54       233
           1       0.31      0.43      0.36       126

    accuracy                           0.47       359
   macro avg       0.46      0.46      

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### Equalized Odds

In [9]:
add_contraint(model, 'EO', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Exponentiated Gradient Reduction Alg is used here with  EO  as the fairness constraint.

Evaluation of  EO -constrained classifier overall:
[[ 602  247]
 [ 292 1859]]
              precision    recall  f1-score   support

           0       0.67      0.71      0.69       849
           1       0.88      0.86      0.87      2151

    accuracy                           0.82      3000
   macro avg       0.78      0.79      0.78      3000
weighted avg       0.82      0.82      0.82      3000

FNER=FN/(FN+TP)=  0.13575081357508137
FPER=FP/(FP+TN)=  0.29093050647820967
TNR=TN/(TN+FP)=  0.7090694935217904
TPR=TP/(FP+FN)=  0.8642491864249187


Evaluation of  EO -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[163  70]
 [ 17 109]]
              precision    recall  f1-score   support

           0       0.91      0.70      0.79       233
           1       0.61      0.87      0.71       126

    accuracy                           0.76       359
   macro avg       0.76      0.78    

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### True Positive Rate Parity

In [10]:
add_contraint(model, 'TPRP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Exponentiated Gradient Reduction Alg is used here with  TPRP  as the fairness constraint.

Evaluation of  TPRP -constrained classifier overall:
[[ 591  258]
 [ 136 2015]]
              precision    recall  f1-score   support

           0       0.81      0.70      0.75       849
           1       0.89      0.94      0.91      2151

    accuracy                           0.87      3000
   macro avg       0.85      0.82      0.83      3000
weighted avg       0.87      0.87      0.87      3000

FNER=FN/(FN+TP)=  0.06322640632264064
FPER=FP/(FP+TN)=  0.303886925795053
TNR=TN/(TN+FP)=  0.696113074204947
TPR=TP/(FP+FN)=  0.9367735936773594


Evaluation of  TPRP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[114 119]
 [  5 121]]
              precision    recall  f1-score   support

           0       0.96      0.49      0.65       233
           1       0.50      0.96      0.66       126

    accuracy                           0.65       359
   macro avg       0.73      0.72 

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### False Positive Rate Parity

In [11]:
add_contraint(model, 'FPRP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Exponentiated Gradient Reduction Alg is used here with  FPRP  as the fairness constraint.

Evaluation of  FPRP -constrained classifier overall:
[[ 663  186]
 [ 559 1592]]
              precision    recall  f1-score   support

           0       0.54      0.78      0.64       849
           1       0.90      0.74      0.81      2151

    accuracy                           0.75      3000
   macro avg       0.72      0.76      0.73      3000
weighted avg       0.80      0.75      0.76      3000

FNER=FN/(FN+TP)=  0.2598791259879126
FPER=FP/(FP+TN)=  0.21908127208480566
TNR=TN/(TN+FP)=  0.7809187279151943
TPR=TP/(FP+FN)=  0.7401208740120874


Evaluation of  FPRP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[187  46]
 [ 73  53]]
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       233
           1       0.54      0.42      0.47       126

    accuracy                           0.67       359
   macro avg       0.63      0.6

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### Error Rate Parity

In [12]:
add_contraint(model, 'ERP', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Exponentiated Gradient Reduction Alg is used here with  ERP  as the fairness constraint.

Evaluation of  ERP -constrained classifier overall:
[[ 618  231]
 [ 272 1879]]
              precision    recall  f1-score   support

           0       0.69      0.73      0.71       849
           1       0.89      0.87      0.88      2151

    accuracy                           0.83      3000
   macro avg       0.79      0.80      0.80      3000
weighted avg       0.84      0.83      0.83      3000

FNER=FN/(FN+TP)=  0.12645281264528127
FPER=FP/(FP+TN)=  0.27208480565371024
TNR=TN/(TN+FP)=  0.7279151943462897
TPR=TP/(FP+FN)=  0.8735471873547187


Evaluation of  ERP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[221  12]
 [ 52  74]]
              precision    recall  f1-score   support

           0       0.81      0.95      0.87       233
           1       0.86      0.59      0.70       126

    accuracy                           0.82       359
   macro avg       0.83      0.77 

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### Bounded Group Loss (TODO: issue, need to figure out loss parameter)

In [ ]:
#add_contraint(model, 'BGL', 'EG', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

## Grid Search Reduction Alg for Adding Fairness Constraints

### Demographic Parity

In [13]:
add_contraint(model, 'DP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Grid Search Reduction Alg is used here with  DP  as the fairness constraint.

Evaluation of  DP -constrained classifier overall:
[[ 552  297]
 [ 122 2029]]
              precision    recall  f1-score   support

           0       0.82      0.65      0.72       849
           1       0.87      0.94      0.91      2151

    accuracy                           0.86      3000
   macro avg       0.85      0.80      0.82      3000
weighted avg       0.86      0.86      0.86      3000

FNER=FN/(FN+TP)=  0.056717805671780565
FPER=FP/(FP+TN)=  0.3498233215547703
TNR=TN/(TN+FP)=  0.6501766784452296
TPR=TP/(FP+FN)=  0.9432821943282195


Evaluation of  DP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[ 82 151]
 [  1 125]]
              precision    recall  f1-score   support

           0       0.99      0.35      0.52       233
           1       0.45      0.99      0.62       126

    accuracy                           0.58       359
   macro avg       0.72      0.67      0.57     

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### Equalized Odds Used

In [14]:
add_contraint(model, 'EO', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Grid Search Reduction Alg is used here with  EO  as the fairness constraint.

Evaluation of  EO -constrained classifier overall:
[[ 620  229]
 [ 112 2039]]
              precision    recall  f1-score   support

           0       0.85      0.73      0.78       849
           1       0.90      0.95      0.92      2151

    accuracy                           0.89      3000
   macro avg       0.87      0.84      0.85      3000
weighted avg       0.88      0.89      0.88      3000

FNER=FN/(FN+TP)=  0.05206880520688052
FPER=FP/(FP+TN)=  0.2697290930506478
TNR=TN/(TN+FP)=  0.7302709069493521
TPR=TP/(FP+FN)=  0.9479311947931195


Evaluation of  EO -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[209  24]
 [ 30  96]]
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       233
           1       0.80      0.76      0.78       126

    accuracy                           0.85       359
   macro avg       0.84      0.83      0.83      

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### True Positive Rate Parity

In [15]:
add_contraint(model, 'TPRP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Grid Search Reduction Alg is used here with  TPRP  as the fairness constraint.

Evaluation of  TPRP -constrained classifier overall:
[[ 617  232]
 [ 116 2035]]
              precision    recall  f1-score   support

           0       0.84      0.73      0.78       849
           1       0.90      0.95      0.92      2151

    accuracy                           0.88      3000
   macro avg       0.87      0.84      0.85      3000
weighted avg       0.88      0.88      0.88      3000

FNER=FN/(FN+TP)=  0.05392840539284054
FPER=FP/(FP+TN)=  0.27326266195524146
TNR=TN/(TN+FP)=  0.7267373380447585
TPR=TP/(FP+FN)=  0.9460715946071595


Evaluation of  TPRP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[213  20]
 [ 39  87]]
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       233
           1       0.81      0.69      0.75       126

    accuracy                           0.84       359
   macro avg       0.83      0.80      0.8

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### False Positive Rate Parity

In [16]:
add_contraint(model, 'FPRP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Grid Search Reduction Alg is used here with  FPRP  as the fairness constraint.

Evaluation of  FPRP -constrained classifier overall:
[[ 617  232]
 [ 188 1963]]
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       849
           1       0.89      0.91      0.90      2151

    accuracy                           0.86      3000
   macro avg       0.83      0.82      0.82      3000
weighted avg       0.86      0.86      0.86      3000

FNER=FN/(FN+TP)=  0.08740120874012088
FPER=FP/(FP+TN)=  0.27326266195524146
TNR=TN/(TN+FP)=  0.7267373380447585
TPR=TP/(FP+FN)=  0.9125987912598791


Evaluation of  FPRP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[233   0]
 [122   4]]
              precision    recall  f1-score   support

           0       0.66      1.00      0.79       233
           1       1.00      0.03      0.06       126

    accuracy                           0.66       359
   macro avg       0.83      0.52      0.4

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### Error Rate Parity

In [17]:
add_contraint(model, 'ERP', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)

Grid Search Reduction Alg is used here with  ERP  as the fairness constraint.

Evaluation of  ERP -constrained classifier overall:
[[ 607  242]
 [ 122 2029]]
              precision    recall  f1-score   support

           0       0.83      0.71      0.77       849
           1       0.89      0.94      0.92      2151

    accuracy                           0.88      3000
   macro avg       0.86      0.83      0.84      3000
weighted avg       0.88      0.88      0.88      3000

FNER=FN/(FN+TP)=  0.056717805671780565
FPER=FP/(FP+TN)=  0.2850412249705536
TNR=TN/(TN+FP)=  0.7149587750294464
TPR=TP/(FP+FN)=  0.9432821943282195


Evaluation of  ERP -constrained classifier by race:
EVALUATION FOR BLACK GROUP
[[223  10]
 [ 56  70]]
              precision    recall  f1-score   support

           0       0.80      0.96      0.87       233
           1       0.88      0.56      0.68       126

    accuracy                           0.82       359
   macro avg       0.84      0.76      0.78  

The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


### Bounded Group Loss (TODO: issue, need to figure out loss parameter)

In [ ]:
#add_contraint(model, 'BGL', 'GS', X_train, y_train, race_train, race_test, X_test, y_test, y_predict, sample_weight_test, False)